# HW6, DATSCI W261

Team: Kuan Lin, Alejandro J. Rojas, Ricardo Barrera<br/>
Emails: kuanlin@ischool.berkeley.edu, ale@ischool.berkeley.edu, ricardofrank@ischool.berkeley.edu<br/>
Time of Initial Submission: 8:00 AM PST, Thursday, March 19, 2016<br/>
W261-1, Spring 2016 Week 9 Homework

# HW 9.0: Short answer questions

What is PageRank and what is it used for in the context of web search?
What modifications have to be made to the webgraph in order to leverage the machinery of Markov Chains to 
compute the steady stade distibuton?<br/>
OPTIONAL: In topic-specific pagerank, how can we insure that the irreducible property is satified? (HINT: see HW9.4)

#### Answer:

PageRank is a way to rank importance of nodes in a network.  By modeling the transition within a network as a Markov process, PageRank can be derived by finding the left eigenvector of the transition matrix, which can be interpreted as the steady-state probability of each node.  Nodes with higher steady-state probability is therefore ranked higher.  In the context of web search, the 'transition' can be interpreted as a person surfing among many webpages, and the resulting steady-state probability is the probability of the surfer visiting a specific page.  In addition, the algorithm needs the following modifications before applying to webgraphs:

- Redistribution of weights from dangling nodes (nodes with no outlinks).  This is to ensure total probabilities sum up to one.
- Teleportation: This is a small probability that a user would randomly jump to another webpage rather than via following a link.

# HW 9.1: MRJob implementation of basic PageRank

Write a basic MRJob implementation of the iterative PageRank algorithm
that takes sparse adjacency lists as input (as explored in HW 7).
Make sure that you implementation utilizes teleportation (1-damping/the number of nodes in the network), 
and further, distributes the mass of dangling nodes with each iteration
so that the output of each iteration is correctly normalized (sums to 1).
[NOTE: The PageRank algorithm assumes that a random surfer (walker), starting from a random web page,
chooses the next page to which it will move by clicking at random, with probability d,
one of the hyperlinks in the current page. This probability is represented by a so-called
‘damping factor’ d, where d ∈ (0, 1). Otherwise, with probability (1 − d), the surfer
jumps to any web page in the network. If a page is a dangling end, meaning it has no
outgoing hyperlinks, the random surfer selects an arbitrary web page from a uniform
distribution and “teleports” to that page]


As you build your code, use the test data

s3://ucb-mids-mls-networks/PageRank-test.txt
Or under the Data Subfolder for HW7 on Dropbox with the same file name. 
(On Dropbox https://www.dropbox.com/sh/2c0k5adwz36lkcw/AAAAKsjQfF9uHfv-X9mCqr9wa?dl=0)

with teleportation parameter set to 0.15 (1-d, where d, the damping factor is set to 0.85), and crosscheck
your work with the true result, displayed in the first image
in the Wikipedia article:

https://en.wikipedia.org/wiki/PageRank

and here for reference are the corresponding PageRank probabilities:

<pre>
A,0.033
B,0.384
C,0.343
D,0.039
E,0.081
F,0.039
G,0.016
H,0.016
I,0.016
J,0.016
K,0.016
</pre>

In [76]:
%%writefile pageRank.py
from mrjob.job import MRJob
from mrjob.step import MRStep
#from mrjob.protocol import ReprProtocol
#from mrjob.protocol import RawProtocol

class PageRank(MRJob):
    
    #OUTPUT_PROTOCOL = ReprProtocol
    
    def configure_options(self):
        super(PageRank, self).configure_options()

        self.add_passthrough_option(
            '--prob-norm-factor', dest='prob_norm_factor', default=0.0, type='float',
            help='normalization factor for node weights')

        self.add_passthrough_option(
            '--damping-factor', dest='damping_factor', default=0.85,
            type='float',
            help='probability a web surfer will continue clicking on links')
        
        self.add_passthrough_option(
            '--total-nodes', dest='total_nodes', default=-1,
            type='int',
            help='total number of nodes in the graph')
        
        self.add_passthrough_option(
            '--loss-mass', dest='loss_mass', default=0.0,
            type='float',
            help='loss mass that needs to be redistributed')
        
        self.add_passthrough_option(
            '--counter-precision', dest='counter_precision', default=6,
            type='int',
            help='number of decimal points to preserve when using counter to store floats')
        
        self.add_passthrough_option(
            '--prank-chg-thold', dest='prank_chg_thold', default=0.001,
            type='float',
            help='threshold for change in PageRank value to continue iteration')
        
        self.add_passthrough_option(
            '--is-final-iter', dest='is_final_iter', default=0,
            type='int',
            help='1 indicates final iteration which needs to apply just the mapper.')
    
    def sendScores(self, _, line):
        # Mapper, send score to each outline
        
        if self.options.total_nodes == -1:
            # accoumate total node numbers
            self.increment_counter('prank', 'total_nodes', 1)
        
        lineArr = line.strip().split('\t')
        
        if '[' not in line: # original input
            # first iteration, don't have total node numbers yet.
            # initialize pageRank to 1.0, and will have to normalize later
            node_weight = 1.0
            node_id = lineArr[0]
            outlinks = eval(lineArr[1])
            self.increment_counter('prank', 'prod_norm_factor', 1)
        else:
            node_id = lineArr[0].replace('"', '')
            payload = eval(lineArr[1])
            outlinks = payload[0]
            node_weight = payload[1]
            if self.options.prob_norm_factor > 0: # need to re-normalize weights so they sum up to 1
                node_weight = node_weight / self.options.prob_norm_factor
                
            if self.options.total_nodes != -1:
                # accounts for teleportation and dangling nodes
                node_weight = (1-self.options.damping_factor)*(1.0/self.options.total_nodes) + self.options.damping_factor*(self.options.loss_mass/self.options.total_nodes + node_weight)
        
        if self.options.is_final_iter == 1:
            yield node_id, (outlinks, round(node_weight, 4))
        else:
            yield node_id, ('node', outlinks, node_weight) # original node info
            if len(outlinks) == 0:
                # dangling nodes, increment loss mass counter
                self.increment_counter('prank', 'loss_mass', int(node_weight*(10**self.options.counter_precision)))
            else:
                for out_node in outlinks:
                    yield out_node, ('score', 1.0*node_weight/len(outlinks), self.options.counter_precision) # new score
                    
    def sendScoreCombinder(self, nodeId, payload):
        total_new_score = 0.0
        for data in payload:
            if data[0] == 'node':
                yield nodeId, data
            else:
                total_new_score += data[1]
        if total_new_score > 0:
            yield nodeId, ('score', total_new_score)
                
    def receive_scores(self, nodeId, payload):
        prev_score = 0.0
        outlinks = {}
        total_new_score = 0.0
        
        for data in payload:
            if data[0] == 'node':
                outlinks = data[1]
                prev_score = data[2]
            else:
                total_new_score += data[1]
                      
        if abs(total_new_score-prev_score) > self.options.prank_chg_thold:
            self.increment_counter('prank', 'chg_over_thold', 1)
        
        yield nodeId, (outlinks, round(total_new_score, 4))
        
    def steps(self):
        if self.options.is_final_iter == 1:
            return ([MRStep(mapper=self.sendScores)])
        else:
            return ([MRStep(mapper=self.sendScores, reducer=self.receive_scores)])
        

Overwriting pageRank.py


#### Page Rank Driver

In [79]:
%load_ext autoreload
%autoreload 2
from pageRank import PageRank

output_dir = 'C:\\Users\\kuanlin\\Desktop\\Berkeley_MIDS\\courses\\W261_ML_at_Scale\\ww9\\PageRankOutputs\\iter'

# first iteration, collect normalization factors
mr_job = PageRank(args=['PageRank-test.txt', '--no-output', '--output-dir='+output_dir+'1'])
with mr_job.make_runner() as runner: 
    runner.run()
    norm_factor = runner.counters()[0]['prank']['prod_norm_factor']
    
# second iteration, normalize initialization values, and collect total nodes
mr_job = PageRank(args=[output_dir+'1', '--no-output', '--output-dir='+output_dir+'2', '--prob-norm-factor='+str(norm_factor)])
with mr_job.make_runner() as runner: 
    runner.run()
    total_nodes = runner.counters()[0]['prank']['total_nodes']
    loss_mass = 0.0
    if 'prank' in runner.counters()[0] and 'loss_mass' in runner.counters()[0]['prank']:
        loss_mass = 1.0*runner.counters()[0]['prank']['loss_mass']/(10**mr_job.options.counter_precision)

# iter until no changes or max iter reached
iter_num = 2
max_iter = 20
while(1):
    iter_num += 1
    
    if iter_num == max_iter or ('prank' in runner.counters()[0] and 'chg_over_thold' not in runner.counters()[0]['prank']):
        final_iter = True
    else:
        final_iter = False
    
    if not final_iter:
        mr_job = PageRank(args=[output_dir+str(iter_num-1), '--no-output', '--output-dir='+output_dir+str(iter_num), 
                '--total-nodes='+str(total_nodes), '--loss-mass='+str(loss_mass)])
    else:
        mr_job = PageRank(args=[output_dir+str(iter_num-1), '--no-output', '--output-dir='+output_dir+str(iter_num), 
                '--total-nodes='+str(total_nodes), '--loss-mass='+str(loss_mass), '--is-final-iter=1'])
    with mr_job.make_runner() as runner: 
        runner.run()
        loss_mass = 0.0
        if 'prank' in runner.counters()[0] and 'loss_mass' in runner.counters()[0]['prank']:
            loss_mass = 1.0*runner.counters()[0]['prank']['loss_mass']/(10**mr_job.options.counter_precision)
    
    if final_iter:
        break
    

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [80]:
# show the result:
output_file = output_dir + str(iter_num) + '\\part-00000'
for line in open(output_file, 'r'): print line.strip()

"A"	[{}, 0.0328]
"B"	[{"C": 1}, 0.3924]
"C"	[{"B": 1}, 0.3351]
"D"	[{"A": 1, "B": 1}, 0.0391]
"E"	[{"B": 1, "D": 1, "F": 1}, 0.0809]
"F"	[{"B": 1, "E": 1}, 0.0391]
"G"	[{"B": 1, "E": 1}, 0.0162]
"H"	[{"B": 1, "E": 1}, 0.0162]
"I"	[{"B": 1, "E": 1}, 0.0162]
"J"	[{"E": 1}, 0.0162]
"K"	[{"E": 1}, 0.0162]


In [96]:
%%writefile topNPageRanks.py

# MRJob code to grab topN pageRanks

from mrjob.job import MRJob

class topNPageRanks(MRJob):
    
    def configure_options(self):
        super(topNPageRanks, self).configure_options()

        self.add_passthrough_option(
            '--top-n', dest='top_n', default=100, type='int',
            help='select top N page ranks')
    
    def mapper(self, _, line):
        lineArr = line.strip().split('\t')
        nodeId = lineArr[0].replace('"', '')
        rank_val = eval(lineArr[1])[1]
        yield None, (rank_val, nodeId)
        
    def combiner(self, _, ranked_nodes):
        top_n_nodes = []
        for node in ranked_nodes:
            if len(top_n_nodes) < self.options.top_n:
                top_n_nodes.append(node)
                top_n_nodes = sorted(top_n_nodes, reverse=True)
            elif node[0] > top_n_nodes[self.options.top_n-1][0]:
                top_n_nodes.append(node)
                top_n_nodes = sorted(top_n_nodes, reverse=True)[:self.options.top_n]
        for node in top_n_nodes:
            yield None, node
        
    def reducer(self, _, ranked_nodes):
        top_n_nodes = []
        for node in ranked_nodes:
            if len(top_n_nodes) < self.options.top_n:
                top_n_nodes.append(node)
                top_n_nodes = sorted(top_n_nodes, reverse=True)
            elif node[0] > top_n_nodes[self.options.top_n-1][0]:
                top_n_nodes.append(node)
                top_n_nodes = sorted(top_n_nodes, reverse=True)[:self.options.top_n]
        for node in top_n_nodes:
            yield node[1], node[0]

if __name__ == '__main__':
    topNPageRanks.run()

Overwriting topNPageRanks.py


In [98]:
!python topNPageRanks.py C:\\Users\\kuanlin\\Desktop\\Berkeley_MIDS\\courses\\W261_ML_at_Scale\\ww9\\PageRankOutputs\\iter20 --top-n=5

"B"	0.3924
"C"	0.3351
"E"	0.0809
"F"	0.0391
"D"	0.0391


no configs found; falling back on auto-configuration
no configs found; falling back on auto-configuration
creating tmp directory c:\temp\topNPageRanks.kuanlin.20160314.070605.221000

PLEASE NOTE: Starting in mrjob v0.5.0, protocols will be strict by default. It's recommended you run your job with --strict-protocols or set up mrjob.conf as described at https://pythonhosted.org/mrjob/whats-new.html#ready-for-strict-protocols

writing to c:\temp\topNPageRanks.kuanlin.20160314.070605.221000\step-0-mapper_part-00000
Counters from step 1:
  (no counters found)
writing to c:\temp\topNPageRanks.kuanlin.20160314.070605.221000\step-0-mapper-sorted
> sort 'c:\temp\topNPageRanks.kuanlin.20160314.070605.221000\step-0-mapper_part-00000'
writing to c:\temp\topNPageRanks.kuanlin.20160314.070605.221000\step-0-reducer_part-00000
Counters from step 1:
  (no counters found)
Moving c:\temp\topNPageRanks.kuanlin.20160314.070605.221000\step-0-reducer_part-00000 -> c:\temp\topNPageRanks.kuanlin.20160314.07060